Lien du jeu de données : https://explore.data.gouv.fr/fr/datasets/586dae65a3a7290df6f4be90/?Code%20de%20la%20discipline__exact=disc06&Ann%C3%A9e__exact=2020#/resources/c7c9642b-9fa1-40a0-83d5-1615c15b4178 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame
import textwrap
import warnings
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler

# Désactiver les FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def load_data(path: str = "data.csv") -> DataFrame:
    return pd.read_csv(path, delimiter=";")

In [3]:
df = load_data()

In [7]:
df.head()

,Année,Diplôme,situation,Genre,Disciplines,Code du domaine,Domaine,Code de la discipline,Discipline,Code du secteur disciplinaire,...,Agriculteur,"Artisan, commerçant, chef d'entreprise",Profession libérale,Personnel de catégorie A de la fonction publique,"Ingénieur, cadre, prof. libérales, prof. intellectuelles sup",Personnel de catégorie B de la fonction publique,"Emploi de niveau intermédiaire : technicien, agent de maîtrise…",Personnel de catégorie C de la fonction publique,"Manœuvre, ouvrier","Employé de bureau, de commerce, personnel de service"
0,2014,MASTER LMD,30 mois après le diplôme,femmes,"Droit, économie et gestion>Autres formations j...",DEG,"Droit, économie et gestion",disc05,"Autres formations juridiques, économiques et d...",disc05_02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,MASTER LMD,18 mois après le diplôme,hommes,"Droit, économie et gestion>Autres formations j...",DEG,"Droit, économie et gestion",disc05,"Autres formations juridiques, économiques et d...",disc05_02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,MASTER LMD,30 mois après le diplôme,hommes,"Droit, économie et gestion>Autres formations j...",DEG,"Droit, économie et gestion",disc05,"Autres formations juridiques, économiques et d...",disc05_02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,MASTER LMD,30 mois après le diplôme,femmes,"Droit, économie et gestion>Autres formations j...",DEG,"Droit, économie et gestion",disc05,"Autres formations juridiques, économiques et d...",disc05_02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,MASTER LMD,18 mois après le diplôme,hommes,"Droit, économie et gestion>Autres formations j...",DEG,"Droit, économie et gestion",disc05,"Autres formations juridiques, économiques et d...",disc05_02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3267 entries, 0 to 3266
Data columns (total 77 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Année                                                                  3267 non-null   int64  
 1   Diplôme                                                                3267 non-null   object 
 2   situation                                                              3267 non-null   object 
 3   Genre                                                                  3267 non-null   object 
 4   Disciplines                                                            3267 non-null   object 
 5   Code du domaine                                                        3267 non-null   object 
 6   Domaine                                                                3267 non-null   o

In [15]:
df.shape

(3267, 77)

# DATAMANAGEMENT

In [17]:
df_clean = df

### Filtrage des données

Garder uniquement les lignes où le genre est "femmes et hommes" pour que l'information ne soit pas redondante

In [19]:
df_fh =df_clean[df_clean["Genre"] == "femmes et hommes"]

Supprimer la colonne Genre car variable constante

In [21]:
df_fh = df_fh.drop(columns = "Genre")

In [23]:
df_fh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1185 entries, 6 to 3266
Data columns (total 76 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Année                                                                  1185 non-null   int64  
 1   Diplôme                                                                1185 non-null   object 
 2   situation                                                              1185 non-null   object 
 3   Disciplines                                                            1185 non-null   object 
 4   Code du domaine                                                        1185 non-null   object 
 5   Domaine                                                                1185 non-null   object 
 6   Code de la discipline                                                  1185 non-null   object

Remplacer les nd et ns en données manquantes

In [25]:
df_fh.replace(["ns", "nd", "na"], np.nan, inplace=True)

Suite à cela, le nombre de données manquantes dans la variable à expliquer Taux d'insertion passe à 462 au lieu de 14

In [27]:
print(df_fh['Taux d’insertion'].isnull().sum())

86


In [29]:
df_fh.dropna(axis=0, subset='Taux d’insertion', inplace=True)

In [31]:
print(df_fh['Taux d’insertion'].isnull().sum())

0


## Gestion des données manquantes

In [33]:
print(df_fh.isna().mean().mul(100).sort_values(ascending=False).to_string())

Taux d'emploi salarié en France                                          90.172884
Taux d'emploi                                                            90.172884
Contrat spécifique au doctorat                                           86.897179
Organisation internationale ou une institution de l'Union européenne     86.624204
Société d'économie mixte                                                 86.624204
Activités immobilières                                                   86.624204
Agriculture, sylviculture et pêche                                       79.799818
Activités financières et d’assurance                                     79.708826
Commerce, transports, héberg-ement et restauration                       79.708826
Une entreprise privée                                                    79.708826
Information et communication                                             79.708826
Industries (manufacturières,  extractives et autres)                     79.708826
Cons

Supprimer les variables avec taux de données manquantes très importants (>50%)

In [35]:
df_fh.columns

Index(['Année', 'Diplôme', 'situation', 'Disciplines', 'Code du domaine',
       'Domaine', 'Code de la discipline', 'Discipline',
       'Code du secteur disciplinaire', 'Secteur disciplinaire',
       'Nombre de réponses', 'Taux d’insertion', 'Taux d'emploi',
       'Taux d'emploi salarié en France',
       'Part des emplois de niveau cadre ou profession intermédiaire',
       'Part des emplois de niveau cadre',
       '% emplois extérieurs à la région de l’université',
       'Part des emplois stables', 'Part des emplois à temps plein',
       'Salaire net mensuel médian des emplois à temps plein',
       'Salaire brut annuel estimé',
       'Premier quartile des salaires nets mensuels des emplois à temps plein',
       'Dernier quartile des salaires nets mensuels des emplois à temps plein',
       'Part des diplômés boursiers dans la discipline', 'Part des femmes',
       'Taux de chômage national', 'Salaire net mensuel médian national',
       'Code du secteur disciplinaire SISE',

In [37]:
# Calculer le seuil de NaN pour chaque colonne
seuil_na = len(df_fh) * 0.5  # 50% de NaN

# Supprimer les colonnes avec plus de 50% de NaN
df_fh = df_fh.dropna(axis=1, thresh=seuil_na)

In [39]:
df_fh.columns

Index(['Année', 'Diplôme', 'situation', 'Disciplines', 'Code du domaine',
       'Domaine', 'Code de la discipline', 'Discipline',
       'Code du secteur disciplinaire', 'Secteur disciplinaire',
       'Nombre de réponses', 'Taux d’insertion',
       'Part des emplois de niveau cadre ou profession intermédiaire',
       'Part des emplois de niveau cadre',
       '% emplois extérieurs à la région de l’université',
       'Part des emplois stables', 'Part des emplois à temps plein',
       'Salaire net mensuel médian des emplois à temps plein',
       'Salaire brut annuel estimé',
       'Premier quartile des salaires nets mensuels des emplois à temps plein',
       'Dernier quartile des salaires nets mensuels des emplois à temps plein',
       'Part des diplômés boursiers dans la discipline', 'Part des femmes',
       'Taux de chômage national', 'Salaire net mensuel médian national',
       'Code du secteur disciplinaire SISE',
       'Salaire net mensuel national 1er quartile',
      

Supprimer les colonnes "codes" car peu utiles

In [41]:
list_codes = ['Code du domaine',  'Code de la discipline', 
       'Code du secteur disciplinaire', 'Code du secteur disciplinaire SISE', 'cle_DISC']


df_fh = df_fh.drop(columns = list_codes ) 

### Conversion de variables

In [43]:
#Conversion de object vers float64

list_object = ['Diplôme', 'situation', 'Genre', 'Disciplines',
 'Domaine', 'Discipline', 'Secteur disciplinaire']

list_columns = df_fh.columns

liste_num = [x for x in list_columns if x not in list_object]

for i in liste_num: 
    df_fh[i] = df_fh[i].astype("float64")



Remplissage des données manquantes

In [45]:
print(df_fh.isna().mean().mul(100).sort_values(ascending=False).to_string())

Dernier quartile des salaires nets mensuels des emplois à temps plein    13.830755
Premier quartile des salaires nets mensuels des emplois à temps plein    13.830755
Salaire net mensuel national 3ème quartile                                5.004550
Salaire net mensuel national 1er quartile                                 5.004550
Salaire net mensuel médian des emplois à temps plein                      2.820746
Salaire brut annuel estimé                                                2.547771
Nombre de réponses                                                        1.455869
Part des emplois de niveau cadre                                          0.818926
Part des emplois à temps plein                                            0.818926
Part des emplois de niveau cadre ou profession intermédiaire              0.818926
Part des emplois stables                                                  0.636943
% emplois extérieurs à la région de l’université                          0.545951
Part

Nous allons imputer les variables par la médiane après avoir fait un group by sur la situation,l'année et le secteur disciplinaire

In [47]:
# 2. Variables numériques (utilisation de la médiane)
vars_na = ['Part des emplois de niveau cadre ou profession intermédiaire',
       'Part des emplois de niveau cadre',
       '% emplois extérieurs à la région de l’université',
       'Part des emplois stables', 'Part des emplois à temps plein',
       'Salaire net mensuel médian des emplois à temps plein',
       'Salaire brut annuel estimé',
       'Premier quartile des salaires nets mensuels des emplois à temps plein',
       'Dernier quartile des salaires nets mensuels des emplois à temps plein',
       'Part des diplômés boursiers dans la discipline', 'Part des femmes', 
       'Salaire net mensuel national 1er quartile',
       'Salaire net mensuel national 3ème quartile', "Nombre de réponses"]
    
for var in vars_na:
    # Imputation conditionnelle par secteur et année
    df_fh[var] = df_fh.groupby(['Domaine','Année', 'situation'])[var].transform(
        lambda x: x.fillna(x.median()))

In [49]:
print(df_fh.isna().mean().mul(100).sort_values(ascending=False).to_string())

Salaire net mensuel national 3ème quartile                               5.004550
Salaire net mensuel national 1er quartile                                5.004550
Nombre de réponses                                                       0.181984
Part des emplois à temps plein                                           0.000000
Salaire net mensuel médian national                                      0.000000
Taux de chômage national                                                 0.000000
Part des femmes                                                          0.000000
Part des diplômés boursiers dans la discipline                           0.000000
Dernier quartile des salaires nets mensuels des emplois à temps plein    0.000000
Premier quartile des salaires nets mensuels des emplois à temps plein    0.000000
Salaire brut annuel estimé                                               0.000000
Salaire net mensuel médian des emplois à temps plein                     0.000000
Année           

In [51]:
#Imputation des salaires 1er et 3ème quartile en 2011 avec la valeur de 2012

df_fh['Salaire net mensuel national 1er quartile'] = df_fh['Salaire net mensuel national 1er quartile'].fillna(1460)

df_fh['Salaire net mensuel national 3ème quartile'] =df_fh['Salaire net mensuel national 1er quartile'].fillna(2230)

In [53]:
#Imputation pour le nombre de réponses par la médiane globale
df_fh["Nombre de réponses"] = df_fh.groupby('Année')['Nombre de réponses'].transform(lambda x: x.fillna(x.median()))

In [55]:
print(df_fh.isna().mean().mul(100).sort_values(ascending=False).to_string())

Année                                                                    0.0
Diplôme                                                                  0.0
Salaire net mensuel national 1er quartile                                0.0
Salaire net mensuel médian national                                      0.0
Taux de chômage national                                                 0.0
Part des femmes                                                          0.0
Part des diplômés boursiers dans la discipline                           0.0
Dernier quartile des salaires nets mensuels des emplois à temps plein    0.0
Premier quartile des salaires nets mensuels des emplois à temps plein    0.0
Salaire brut annuel estimé                                               0.0
Salaire net mensuel médian des emplois à temps plein                     0.0
Part des emplois à temps plein                                           0.0
Part des emplois stables                                                 0.0

In [57]:
df_fh.describe()

,Année,Nombre de réponses,Taux d’insertion,Part des emplois de niveau cadre ou profession intermédiaire,Part des emplois de niveau cadre,% emplois extérieurs à la région de l’université,Part des emplois stables,Part des emplois à temps plein,Salaire net mensuel médian des emplois à temps plein,Salaire brut annuel estimé,Premier quartile des salaires nets mensuels des emplois à temps plein,Dernier quartile des salaires nets mensuels des emplois à temps plein,Part des diplômés boursiers dans la discipline,Part des femmes,Taux de chômage national,Salaire net mensuel médian national,Salaire net mensuel national 1er quartile,Salaire net mensuel national 3ème quartile
count,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000
mean,2015.715196,1795.676979,88.100091,85.641037,61.903549,44.070974,65.574613,91.492721,1865.987261,29329.026388,1600.850773,2199.217470,32.611010,56.863967,8.529936,1892.866242,1479.090082,1479.090082
std,2.692812,4386.218406,5.783668,10.532245,17.435055,10.038200,15.720474,8.307772,254.775793,4080.523072,221.972199,278.054287,6.488743,21.375639,1.063377,62.883587,20.966842,20.966842
min,2011.000000,27.000000,61.000000,40.000000,10.000000,7.000000,21.000000,56.000000,1350.000000,21100.000000,1190.000000,1590.000000,11.000000,4.000000,6.900000,1800.000000,1450.000000,1450.000000
25%,2013.000000,184.000000,85.000000,79.000000,49.000000,39.000000,54.000000,87.000000,1680.000000,26350.000000,1420.000000,2000.000000,29.000000,42.000000,7.800000,1830.000000,1460.000000,1460.000000
50%,2016.000000,447.000000,89.000000,87.000000,60.000000,44.000000,65.000000,95.000000,1830.000000,28900.000000,1600.000000,2200.000000,32.000000,60.000000,8.500000,1870.000000,1480.000000,1480.000000
75%,2018.000000,1399.000000,92.000000,95.000000,76.000000,51.000000,78.000000,98.000000,2050.000000,32400.000000,1760.000000,2400.000000,36.500000,73.000000,9.700000,1930.000000,1490.000000,1490.000000
max,2020.000000,32157.000000,100.000000,100.000000,100.000000,77.000000,97.000000,100.000000,2780.000000,43400.000000,2300.000000,3350.000000,50.000000,92.000000,10.100000,2020.000000,1530.000000,1530.000000


### Encodage des variables qualitatives

In [59]:
df_fh.columns

Index(['Année', 'Diplôme', 'situation', 'Disciplines', 'Domaine', 'Discipline',
       'Secteur disciplinaire', 'Nombre de réponses', 'Taux d’insertion',
       'Part des emplois de niveau cadre ou profession intermédiaire',
       'Part des emplois de niveau cadre',
       '% emplois extérieurs à la région de l’université',
       'Part des emplois stables', 'Part des emplois à temps plein',
       'Salaire net mensuel médian des emplois à temps plein',
       'Salaire brut annuel estimé',
       'Premier quartile des salaires nets mensuels des emplois à temps plein',
       'Dernier quartile des salaires nets mensuels des emplois à temps plein',
       'Part des diplômés boursiers dans la discipline', 'Part des femmes',
       'Taux de chômage national', 'Salaire net mensuel médian national',
       'Salaire net mensuel national 1er quartile',
       'Salaire net mensuel national 3ème quartile'],
      dtype='object')

In [61]:
df_fh = df_fh.drop(columns = ["Nombre de réponses", "Diplôme"])

In [65]:
#garder Année en format interger
df_fh['Année'] = df_fh['Année'].astype(int)


#encoder la variable Situation en 0 ou 1 
label_encoder = LabelEncoder()

df_fh["situation"] = label_encoder.fit_transform(df_fh["situation"])


# colonnes à encoder 
liste_encoder = ["Disciplines", "Domaine", "Discipline", "Secteur disciplinaire"]

# Encodage (selon la fréquence)
for col in liste_encoder:
    # Calcul de la fréquence pour chaque catégorie dans la colonne
    frequency_encoding = df_fh[col].value_counts() / len(df_fh)
    
    # Remplacer les catégories par leur fréquence
    df_fh[col + '_encoded'] = df_fh[col].map(frequency_encoding)


df_fh_encoded = df_fh.drop(columns = df_fh[liste_encoder])


In [67]:
df_fh_encoded.head()

,Année,situation,Taux d’insertion,Part des emplois de niveau cadre ou profession intermédiaire,Part des emplois de niveau cadre,% emplois extérieurs à la région de l’université,Part des emplois stables,Part des emplois à temps plein,Salaire net mensuel médian des emplois à temps plein,Salaire brut annuel estimé,...,Part des diplômés boursiers dans la discipline,Part des femmes,Taux de chômage national,Salaire net mensuel médian national,Salaire net mensuel national 1er quartile,Salaire net mensuel national 3ème quartile,Disciplines_encoded,Domaine_encoded,Discipline_encoded,Secteur disciplinaire_encoded
6,2019,1,89.0,88.0,61.0,8.0,81.0,97.0,2160.0,33600.0,...,44.0,67.0,7.2,1980.0,1480.0,1480.0,0.007279,0.154686,0.068244,0.007279
11,2017,0,96.0,87.0,58.0,44.0,79.0,98.0,2080.0,32500.0,...,24.0,55.0,7.8,1900.0,1470.0,1470.0,0.017288,0.154686,0.068244,0.017288
13,2013,1,93.0,89.0,63.0,20.0,87.0,99.0,2240.0,35000.0,...,19.0,50.0,9.9,1870.0,1490.0,1490.0,0.017288,0.154686,0.068244,0.017288
14,2018,0,93.0,91.0,55.0,50.0,77.0,97.0,2160.0,33700.0,...,38.0,54.0,7.8,1930.0,1490.0,1490.0,0.017288,0.154686,0.068244,0.017288
18,2016,1,96.0,90.0,62.0,38.0,88.0,99.0,2260.0,35300.0,...,23.0,46.0,8.5,1830.0,1450.0,1450.0,0.017288,0.154686,0.068244,0.017288


### Standardisation

In [69]:
df_fh_encoded.columns

Index(['Année', 'situation', 'Taux d’insertion',
       'Part des emplois de niveau cadre ou profession intermédiaire',
       'Part des emplois de niveau cadre',
       '% emplois extérieurs à la région de l’université',
       'Part des emplois stables', 'Part des emplois à temps plein',
       'Salaire net mensuel médian des emplois à temps plein',
       'Salaire brut annuel estimé',
       'Premier quartile des salaires nets mensuels des emplois à temps plein',
       'Dernier quartile des salaires nets mensuels des emplois à temps plein',
       'Part des diplômés boursiers dans la discipline', 'Part des femmes',
       'Taux de chômage national', 'Salaire net mensuel médian national',
       'Salaire net mensuel national 1er quartile',
       'Salaire net mensuel national 3ème quartile', 'Disciplines_encoded',
       'Domaine_encoded', 'Discipline_encoded',
       'Secteur disciplinaire_encoded'],
      dtype='object')

# Liste variables à standardiser
scaler = StandardScaler()
numeric_columns = df_fh_encoded.select_dtypes(include=['float64']).columns

# Initialiser le scaler
scaler = MinMaxScaler()

df_fh_encoded[numeric_columns] = scaler.fit_transform(df_fh_encoded[numeric_columns])

In [71]:
# Séparation des variables à standardiser
standardiser = ["Salaire net mensuel médian des emplois à temps plein", "Salaire brut annuel estimé", "Premier quartile des salaires nets mensuels des emplois à temps plein",
                    "Dernier quartile des salaires nets mensuels des emplois à temps plein", "Salaire net mensuel médian national",
                    "Salaire net mensuel national 1er quartile", "Salaire net mensuel national 3ème quartile"]

# Standardisation du salaire
scaler = StandardScaler()
# Appliquer la standardisation à toutes les colonnes (même celles normalisées)
df_fh_encoded[standardiser] = scaler.fit_transform(df_fh_encoded[standardiser])

In [73]:
df_fh_encoded.describe()

,Année,situation,Taux d’insertion,Part des emplois de niveau cadre ou profession intermédiaire,Part des emplois de niveau cadre,% emplois extérieurs à la région de l’université,Part des emplois stables,Part des emplois à temps plein,Salaire net mensuel médian des emplois à temps plein,Salaire brut annuel estimé,...,Part des diplômés boursiers dans la discipline,Part des femmes,Taux de chômage national,Salaire net mensuel médian national,Salaire net mensuel national 1er quartile,Salaire net mensuel national 3ème quartile,Disciplines_encoded,Domaine_encoded,Discipline_encoded,Secteur disciplinaire_encoded
count,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1.099000e+03,1.099000e+03,...,1099.000000,1099.000000,1099.000000,1.099000e+03,1.099000e+03,1.099000e+03,1099.000000,1099.000000,1099.000000,1099.000000
mean,2015.715196,0.524113,88.100091,85.641037,61.903549,44.070974,65.574613,91.492721,-1.196091e-16,4.396443e-16,...,32.611010,56.863967,8.529936,-1.390052e-16,-5.076922e-15,-5.076922e-15,0.016341,0.274495,0.094001,0.016398
std,2.692812,0.499646,5.783668,10.532245,17.435055,10.038200,15.720474,8.307772,1.000455e+00,1.000455e+00,...,6.488743,21.375639,1.063377,1.000455e+00,1.000455e+00,1.000455e+00,0.002073,0.122161,0.052316,0.001978
min,2011.000000,0.000000,61.000000,40.000000,10.000000,7.000000,21.000000,56.000000,-2.026182e+00,-2.017578e+00,...,11.000000,4.000000,6.900000,-1.477469e+00,-1.388064e+00,-1.388064e+00,0.000910,0.017288,0.000910,0.000910
25%,2013.000000,0.000000,85.000000,79.000000,49.000000,39.000000,54.000000,87.000000,-7.303360e-01,-7.303923e-01,...,29.000000,42.000000,7.800000,-1.000179e+00,-9.109037e-01,-9.109037e-01,0.017288,0.176524,0.049136,0.017288
50%,2016.000000,1.000000,89.000000,87.000000,60.000000,44.000000,65.000000,95.000000,-1.413150e-01,-1.051879e-01,...,32.000000,60.000000,8.500000,-3.637937e-01,4.341772e-02,4.341772e-02,0.017288,0.219290,0.101001,0.017288
75%,2018.000000,1.000000,92.000000,95.000000,76.000000,51.000000,78.000000,98.000000,7.225824e-01,7.529358e-01,...,36.500000,73.000000,9.700000,5.907847e-01,5.205784e-01,5.205784e-01,0.017288,0.413103,0.120109,0.017288
max,2020.000000,1.000000,100.000000,100.000000,100.000000,77.000000,97.000000,100.000000,3.589151e+00,3.449896e+00,...,50.000000,92.000000,10.100000,2.022652e+00,2.429221e+00,2.429221e+00,0.017288,0.413103,0.176524,0.017288


In [443]:
df_fh_encoded.head()

,Année,situation,Taux d’insertion,Part des emplois de niveau cadre ou profession intermédiaire,Part des emplois de niveau cadre,% emplois extérieurs à la région de l’université,Part des emplois stables,Part des emplois à temps plein,Salaire net mensuel médian des emplois à temps plein,Salaire brut annuel estimé,...,Part des diplômés boursiers dans la discipline,Part des femmes,Taux de chômage national,Salaire net mensuel médian national,Salaire net mensuel national 1er quartile,Salaire net mensuel national 3ème quartile,Disciplines_encoded,Domaine_encoded,Discipline_encoded,Secteur disciplinaire_encoded
6,2019,1,89.0,88.0,61.0,8.0,81.0,97.0,1.154531,1.047150,...,44.0,67.0,7.2,1.386267,0.043418,0.043418,0.007279,0.154686,0.068244,0.007279
11,2017,0,96.0,87.0,58.0,44.0,79.0,98.0,0.840387,0.777454,...,24.0,55.0,7.8,0.113496,-0.433743,-0.433743,0.017288,0.154686,0.068244,0.017288
13,2013,1,93.0,89.0,63.0,20.0,87.0,99.0,1.468676,1.390399,...,19.0,50.0,9.9,-0.363794,0.520578,0.520578,0.017288,0.154686,0.068244,0.017288
14,2018,0,93.0,91.0,55.0,50.0,77.0,97.0,1.154531,1.071667,...,38.0,54.0,7.8,0.590785,0.520578,0.520578,0.017288,0.154686,0.068244,0.017288
18,2016,1,96.0,90.0,62.0,38.0,88.0,99.0,1.547212,1.463953,...,23.0,46.0,8.5,-1.000179,-1.388064,-1.388064,0.017288,0.154686,0.068244,0.017288


Sauvegarde du fichier

In [75]:
df_fh_encoded.to_csv('df_final.csv', index=False)

In [77]:
df_fh_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1099 entries, 6 to 3266
Data columns (total 22 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Année                                                                  1099 non-null   int32  
 1   situation                                                              1099 non-null   int64  
 2   Taux d’insertion                                                       1099 non-null   float64
 3   Part des emplois de niveau cadre ou profession intermédiaire           1099 non-null   float64
 4   Part des emplois de niveau cadre                                       1099 non-null   float64
 5   % emplois extérieurs à la région de l’université                       1099 non-null   float64
 6   Part des emplois stables                                               1099 non-null   float6